In [1]:
import os
import json
import time
import logging
import base64
import requests
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime, timedelta, timezone
from PIL import Image
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.endpoints.orders import OrderCreate, OrderReplace
from oandapyV20.endpoints.positions import PositionDetails
from oandapyV20.endpoints.pricing import PricingInfo
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.orders as orders
from oandapyV20.endpoints.orders import OrderList

import openai
import tiktoken
from oandapyV20.endpoints.pricing import PricingInfo
import csv

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Ensure the existence of the images directory
os.makedirs("images", exist_ok=True)

# CSV file for logging
csv_file = 'trade_logs.csv'
image_folder = 'images'
active_order = None

def append_to_csv(log_data, csv_file):
    # Ensure the CSV file has the correct header if it doesn't exist
    file_exists = os.path.isfile(csv_file)
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=log_data.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(log_data)

def save_image(image_data, filename):
    with open(filename, 'wb') as f:
        f.write(image_data)
        
        
# Parameters for real-time data fetching and processing
granularity = 'M15'  # 15 minutes granularity
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '15 minutes'

# Fetch forex data
def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    rec = {
                        'time': candle.get('time')[0:19],
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    return pd.DataFrame(data)

# Calculate RSI
def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Plot candlestick chart
def plot_candlestick_chart(df, filename):
    df.index = pd.to_datetime(df['time'])
    df.index.name = 'Date'
    
    # Calculate support and resistance
    support = df['low'].rolling(window=20).min()
    resistance = df['high'].rolling(window=20).max()

    # Calculate RSI
    df['RSI'] = calculate_rsi(df['close'])
    
    if df['RSI'].dropna().empty:
        logging.error("RSI calculation resulted in empty data. Skipping plot.")
        return

    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', y_on_right=False)

    addplots = [
        mpf.make_addplot(support, color='blue', linestyle='dashed'),
        mpf.make_addplot(resistance, color='orange', linestyle='dashed'),
        mpf.make_addplot(df['RSI'], panel=1, color='purple', secondary_y=False),
    ]
    
    kwargs = dict(
        type='candle', 
        style=s, 
        addplot=addplots, 
        volume=True, 
        figscale=2.5,  # Further increase the size of the candles
        figratio=(10, 8),  # Adjust figure ratio
        title=pair, 
        ylabel='Price', 
        ylabel_lower='Volume', 
        panel_ratios=(2, 1),  # Allocate more space for the candlestick chart compared to RSI
        tight_layout=True,  # Remove whitespace around the chart
        fontscale=1.2,  # Increase font size for better readability
    )
    
    mpf.plot(df, **kwargs, savefig=dict(fname=filename, dpi=100, pad_inches=0.1))

# Compress image
def compress_image(input_path, output_path, quality=85):
    with Image.open(input_path) as img:
        img = img.convert('RGB')  # Convert to RGB
        img = img.resize((510, 510), Image.LANCZOS)  # Resize to 510x510
        img.save(output_path, 'JPEG', quality=quality)  # Save with reduced quality

# Encode image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Save prompt to txt
def save_prompt_to_txt(prompt_content, filename):
    with open(filename, 'w') as file:
        json.dump(prompt_content, file, indent=4)

# Analyze data with GPT-4o
def analyze_data_with_gpt4o(price_data, image_base64):
    logging.info("Sending data to OpenAI API for analysis")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }

    prompt_content = {
        "content": (
            "Here is the EUR/USD close price data from {price_data['start_date']} to {price_data['end_date']} at a 15-minute interval. Analyze this data and the provided chart to calculate RSI, MACD, Bollinger Bands, and Fibonacci Retracement. Use these indicators, along with the chart, to perform a comprehensive analysis and identify potential trading opportunities. Assign a probability score from 0 to 100 and profit/loss rate for all patterns. Follow these steps:\n"
            "1. Analyze the chart visually and identify possible patterns without considering additional data.\n"
            "2. Using both the chart and calculated indicators, identify possible patterns.\n"
            "3. Analyze the data from the indicators independently and list all possible predictions.\n"
            "4. Combine the results from the three types of analysis (chart-only, chart with data, data-only) to form potential orders. Select the best order based on:\n"
            "   - Majority of detected patterns and predictions.\n"
            "   - Higher benefit-to-loss ratio.\n"
            "   - Predicted profit (minimum 20 to 50 pips).\n"
            "   - Deadline less than one day.\n"
            "5. Verify the selected order with the data and indicators to ensure accuracy.\n"
            "6. If multiple orders are found, only send back the order with the highest confidence and benefit/loss rate.\n"
            "Provide the analysis in JSON format:\n"
            "{\n"
            "   \"order\": {\n"
            "       \"timeframe\": \"15 minutes\",\n"
            "       \"pattern_name\": \"Pattern Name\",\n"
            "       \"confidence_percentage\": xx,\n"
            "       \"action\": \"Buy/Sell\",\n"
            "       \"entry_price\": x.xxxx,\n"
            "       \"take_profit\": x.xxxx,\n"
            "       \"stop_loss\": x.xxxx,\n"
            "       \"profit_loss_ratio\": x.x,\n"
            "       \"deadline_date\": \"yyyy-mm-ddThh:mm:ssZ\"\n"
            "   },\n"
            "   \"best_pattern\": {\n"
            "       \"timeframe\": \"15 minutes\",\n"
            "       \"pattern_name\": \"Pattern Name\",\n"
            "       \"confidence_percentage\": xx,\n"
            "       \"action\": \"Buy/Sell\",\n"
            "       \"entry_price\": x.xxxx,\n"
            "       \"take_profit\": x.xxxx,\n"
            "       \"stop_loss\": x.xxxx,\n"
            "       \"profit_loss_ratio\": x.x,\n"
            "       \"deadline_date\": \"yyyy-mm-ddThh:mm:ssZ\"\n"
            "   }\n"
            "}\n"
        ),
        "price_data": {
            "close_prices": price_data['close_prices']
        },
        "image": f"data:image/png;base64,{image_base64}"
    }

    # Calculate tokens before adding image
    enc = tiktoken.encoding_for_model("gpt-4o")
    num_tokens = len(enc.encode(json.dumps(prompt_content)))
    data_cost = num_tokens * 0.000005
    image_cost = 0.001275  # Fixed image cost based on 512x512 px

    logging.info(f"Data tokens before adding image: {num_tokens}, Estimated Data Cost: ${data_cost:.6f}")

    save_prompt_to_txt(prompt_content, 'final_prompt.txt')

    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    try:
        response_data = response.json()
        return response_data, data_cost, image_cost, prompt_content
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None, data_cost, image_cost, prompt_content

# Extract and place order
def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    content = response_data["choices"][0]["message"]["content"]
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]
    
    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None
    
    orders = analysis.get("orders", [])
    best_pattern = analysis.get("best_pattern", {})
    
    for order in orders:
        if order.get("profit_loss_ratio", 0) > 2:
            logging.info(f"Order Details - Action: {order['action']}, Entry Price: {order['entry_price']}, Take Profit: {order['take_profit']}, Stop Loss: {order['stop_loss']}")
            
            order_details = {
                'action': order['action'],
                'entry_price': order['entry_price'],
                'take_profit': order['take_profit'],
                'stop_loss': order['stop_loss'],
                'deadline_date': order['deadline_date']
            }
            response = place_order(order_details)
            
            if 'orderCancelTransaction' in response:
                logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
            return order_details

    logging.info(f"Best Pattern Details - Action: {best_pattern.get('action', 'N/A')}, Entry Price: {best_pattern.get('entry_price', 'N/A')}, Take Profit: {best_pattern.get('take_profit', 'N/A')}, Stop Loss: {best_pattern.get('stop_loss', 'N/A')}")
    return best_pattern

def place_order(order_details):
    instrument = "EUR_USD"
    
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )
    
    r = OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return {"error": str(e)}



# Save data to CSV
def save_data_to_csv(data, filename):
    data.to_csv(filename, index=False)

# Check if weekend
def is_weekend(date):
    return date.weekday() > 4  # Saturday and Sunday are 5 and 6

# Get last weekday
def get_last_weekday(date):
    while date.weekday() > 4:  # If it's Saturday (5) or Sunday (6)
        date -= timedelta(days=1)
    return date


def get_open_positions(api, account_id, pair):
    # Fetch open positions from the account
    response = api.request(PositionDetails(accountID=account_id, instrument=pair))
    positions = response.get('position', [])
    if isinstance(positions, dict):  # Sometimes it's a single position, convert to list
        positions = [positions]
    return positions

def get_current_price(api, pair):
    # Fetch the current market price
    params = {"instruments": pair}
    response = api.request(PricingInfo(accountID=account_id, params=params))
    return float(response['prices'][0]['closeoutBid'])



def get_orders(api, account_id):
    r = OrderList(accountID=account_id)
    try:
        response = api.request(r)
        return response.get('orders', [])
    except V20Error as e:
        logging.error(f"Failed to fetch orders: {e}")
        return []
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return []
    

def find_order_id(trade_id, orders):
    for order in orders:
        if 'tradeID' in order and order['tradeID'] == trade_id:
            return order['id']
    return None

from oandapyV20.endpoints.orders import OrderReplace



def update_stop_loss(api, account_id, order_id, new_stop_loss):
    data = {
        "order": {
            "type": "STOP_LOSS",
            "price": str(new_stop_loss),
            "timeInForce": "GTC"
        }
    }
    try:
        r = OrderReplace(accountID=account_id, orderID=order_id, data=data)
        response = api.request(r)
        logging.info(f"Stop loss updated for order {order_id} to {new_stop_loss}")
        return response
    except V20Error as e:
        logging.error(f"Failed to update stop loss for order {order_id}: {e}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return None

def monitor_and_manage_orders(api, account_id, pair, trailing_stop_pips=10):
    open_positions = get_open_positions(api, account_id, pair)
    if not open_positions:
        logging.info(f"No open positions for {pair}")
        return

    orders = get_orders(api, account_id)
    if not orders:
        logging.info(f"No orders found for account {account_id}")
        return

    for position in open_positions:
        logging.info(f"Inspecting position: {position}")

        if 'long' in position and position['long']['units'] != '0':
            units = position['long']['units']
            entry_price = float(position['long']['averagePrice'])
            stop_loss = float(position['long']['stopLossOrder']['price']) if 'stopLossOrder' in position['long'] else None
            trade_ids = position['long']['tradeIDs'] if 'tradeIDs' in position['long'] else None
        elif 'short' in position and position['short']['units'] != '0':
            units = position['short']['units']
            entry_price = float(position['short']['averagePrice'])
            stop_loss = float(position['short']['stopLossOrder']['price']) if 'stopLossOrder' in position['short'] else None
            trade_ids = position['short']['tradeIDs'] if 'tradeIDs' in position['short'] else None
        else:
            continue

        if not trade_ids:
            logging.warning(f"Trade IDs not found for position: {position}")
            continue

        trade_id = trade_ids[0]  # Use the first trade ID in the list
        order_id = find_order_id(trade_id, orders)
        if not order_id:
            logging.warning(f"Order ID not found for trade ID: {trade_id}")
            continue

        current_price = get_current_price(api, pair)
        logging.info(f"Monitoring position: Entry Price: {entry_price}, Current Price: {current_price}, Units: {units}")

        # Calculate new stop loss level for trailing stop
        if float(units) > 0:  # Long position
            new_stop_loss = current_price - trailing_stop_pips * 0.0001
            if stop_loss is None or new_stop_loss > stop_loss:
                logging.info(f"Updating stop loss for long position to {new_stop_loss}")
                update_stop_loss(api, account_id, order_id, new_stop_loss)
        else:  # Short position
            new_stop_loss = current_price + trailing_stop_pips * 0.0001
            if stop_loss is None or new_stop_loss < stop_loss:
                logging.info(f"Updating stop loss for short position to {new_stop_loss}")
                update_stop_loss(api, account_id, order_id, new_stop_loss)
# Main execution loop
active_order = None

while True:
    now = datetime.now(timezone.utc)

    if is_weekend(now):
        logging.info("It's the weekend. Using last available weekday data.")
        now = get_last_weekday(now)

    start_time = (now - timedelta(days=3)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = now.strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)
    save_data_to_csv(prices, 'forex_data.csv')

    # Collect all the raw price data
    price_data = {
        "start_date": start_time,
        "end_date": end_time,
        "close_prices": prices['close'].to_list()
    }

    # Generate unique datetime ID
    datetime_id = now.strftime('%Y%m%d%H%M%S')
    filename = f"images/{datetime_id}.png"
    compressed_filename = f"images/{datetime_id}_compressed.jpg"

    # Plot and encode the image
    plot_candlestick_chart(prices, filename)  # Use color image
    compress_image(filename, compressed_filename, quality=85)  # Compress without grayscale
    image_base64 = encode_image(compressed_filename)

    # Analyze data and place orders based on predictions if no active order
    if not active_order:
        analysis_result, data_cost, image_cost, prompt_content = analyze_data_with_gpt4o(price_data, image_base64)
        logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

        if analysis_result:
            response_tokens = analysis_result["usage"]["completion_tokens"]
            response_cost = response_tokens * 0.000015  # Adjust based on actual token cost
            logging.info(f"Response received: {response_tokens} tokens, Cost: ${response_cost:.6f}")

            prompt_tokens = analysis_result["usage"]["prompt_tokens"]
            prompt_cost = prompt_tokens * 0.00005  # Adjust based on actual token cost
            logging.info(f"Prompt sent: {prompt_tokens} tokens, Cost: ${prompt_cost:.6f}")

            order_details = extract_and_place_order(analysis_result)
            if order_details:
                logging.info("Order placed based on analysis.")
                active_order = order_details

                # Save logs to CSV
                log_data = {
                    "date-time-id": datetime_id,
                    "date_time": now.strftime('%Y-%m-%d %H:%M:%S'),
                    "image_path": filename,
                    "response_prompt": json.dumps(analysis_result['choices'][0]['message']['content']),
                    "order_action": order_details['action'],
                    "entry_price": order_details['entry_price'],
                    "take_profit": order_details['take_profit'],
                    "stop_loss": order_details['stop_loss'],
                    "data_cost": data_cost,
                    "image_cost": image_cost,
                    "response_cost": response_cost,
                    "total_cost": data_cost + image_cost + response_cost
                }
                append_to_csv(log_data, csv_file)

    # Order management if there is an active order
    if active_order:
        monitor_and_manage_orders(api, account_id, pair)
        # Check if the order is closed
        open_positions = get_open_positions(api, account_id, pair)
        if not open_positions:
            logging.info(f"Order {active_order} is closed.")
            active_order = None

    logging.info("Waiting for 15 minutes before next run...")
    time.sleep(900)


INFO:root:Fetching forex data from 2024-06-25T20:31:09Z to 2024-06-28T20:31:09Z with granularity M15 for instrument EUR_USD
INFO:root:Sending data to OpenAI API for analysis
INFO:root:Data tokens before adding image: 41364, Estimated Data Cost: $0.206820
INFO:root:OpenAI API Analysis Result: {
    "id": "chatcmpl-9fCXQKNCnAql7H4PGqGELN4LEucxa",
    "object": "chat.completion",
    "created": 1719606672,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "{\n    \"order\": {\n        \"timeframe\": \"15 minutes\",\n        \"pattern_name\": \"Ascending Channel\",\n        \"confidence_percentage\": 85,\n        \"action\": \"Buy\",\n        \"entry_price\": 1.06850,\n        \"take_profit\": 1.07070,\n        \"stop_loss\": 1.06700,\n        \"profit_loss_ratio\": 1.44,\n        \"deadline_date\": \"2023-10-20T15:00:00Z\"\n    },\n    \"best_pattern\": {\n        \"

Check the order placement and get list of the available order

In [7]:
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.endpoints.orders import OrderList, OrderCreate
from oandapyV20.endpoints.positions import OpenPositions
from oandapyV20.endpoints.transactions import TransactionList
from oandapyV20.exceptions import V20Error
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

def place_simple_order(action):
    instrument = "EUR_USD"
    
    units = -10000 if action.upper() == 'SELL' else 10000
    
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=units
    )
    
    r = OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Simple order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing simple order: {e}")
        return {"error": str(e)}
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return {"error": str(e)}

# Example usage to place a simple order:
place_simple_order("BUY")

def fetch_orders(api, account_id):
    try:
        # Fetch orders
        orders_request = OrderList(accountID=account_id)
        orders_response = api.request(orders_request)

        return orders_response['orders']
    except V20Error as e:
        logging.error(f"Error fetching orders: {e}")
        return []

def format_orders(orders):
    data = []

    for order in orders:
        order_data = {
            'Order Number': order['id'],
            'Instrument': order['instrument'],
            'Entry Price': order.get('price'),
            'Stop Loss': order['stopLossOnFill']['price'] if 'stopLossOnFill' in order else None,
            'Take Profit': order['takeProfitOnFill']['price'] if 'takeProfitOnFill' in order else None,
            'Time': order['createTime'],
            'Order Type': order['type'],
            'State': order['state']
        }
        data.append(order_data)

    df = pd.DataFrame(data)
    return df

def fetch_transactions(api, account_id):
    transactions = []
    try:
        # Fetch the first page of transactions
        transactions_request = TransactionList(accountID=account_id, params={"type": "ORDER_FILL"})
        transactions_response = api.request(transactions_request)
        
        if 'transactions' in transactions_response:
            transactions.extend(transactions_response['transactions'])
        else:
            logging.warning("No transactions found in the initial response")
            logging.info(f"Full transactions response: {transactions_response}")

        # Handle pagination if more pages are available
        while 'pages' in transactions_response:
            next_page_url = transactions_response['pages'].get('nextPage')
            if not next_page_url:
                break
            next_page_request = api.request(next_page_url)
            transactions_response = next_page_request
            if 'transactions' in next_page_request:
                transactions.extend(next_page_request['transactions'])
        
        return transactions
    except V20Error as e:
        logging.error(f"Error fetching transactions: {e}")
        return []

# Rest of the code remains the same




def format_transactions(transactions):
    data = []

    for transaction in transactions:
        transaction_data = {
            'Transaction ID': transaction['id'],
            'Instrument': transaction['instrument'],
            'Units': transaction['units'],
            'Price': transaction['price'],
            'P/L': transaction['pl'],
            'Time': transaction['time'],
            'Type': transaction['type']
        }
        data.append(transaction_data)

    df = pd.DataFrame(data)
    return df

def fetch_open_positions(api, account_id):
    try:
        # Fetch open positions
        positions_request = OpenPositions(accountID=account_id)
        positions_response = api.request(positions_request)

        return positions_response['positions']
    except V20Error as e:
        logging.error(f"Error fetching open positions: {e}")
        return []

def format_positions(positions):
    data = []

    for position in positions:
        if 'long' in position and position['long']['units'] != '0':
            units = position['long']['units']
            entry_price = position['long']['averagePrice']
            unrealized_pl = position['long']['unrealizedPL']
        elif 'short' in position and position['short']['units'] != '0':
            units = position['short']['units']
            entry_price = position['short']['averagePrice']
            unrealized_pl = position['short']['unrealizedPL']
        else:
            continue

        position_data = {
            'Instrument': position['instrument'],
            'Units': units,
            'Entry Price': entry_price,
            'Unrealized P/L': unrealized_pl,
        }
        data.append(position_data)

    df = pd.DataFrame(data)
    return df

def display_orders_and_positions():
    orders = fetch_orders(api, account_id)
    orders_df = format_orders(orders)
    transactions = fetch_transactions(api, account_id)
    transactions_df = format_transactions(transactions)
    positions = fetch_open_positions(api, account_id)
    positions_df = format_positions(positions)
    
    print("Orders:")
    print(orders_df)
    print("\nFilled Orders (Transactions):")
    print(transactions_df)
    print("\nOpen Positions:")
    print(positions_df)

# Call the function to display orders, transactions, and positions
display_orders_and_positions()


INFO:root:Simple order placed successfully: {'orderCreateTransaction': {'id': '220', 'accountID': '101-002-7656987-005', 'userID': 7656987, 'batchID': '220', 'requestID': '61255743683133631', 'time': '2024-06-28T20:11:22.004524623Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '10000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '221', 'accountID': '101-002-7656987-005', 'userID': 7656987, 'batchID': '220', 'requestID': '61255743683133631', 'time': '2024-06-28T20:11:22.004524623Z', 'type': 'ORDER_FILL', 'orderID': '220', 'instrument': 'EUR_USD', 'units': '10000', 'requestedUnits': '10000', 'price': '1.07106', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance': '100792.9023', 'gainQuoteHomeConversionFactor': '1.36173710', 'lossQuoteHomeConversionFactor': '1.37542290', 'guaranteedExecutionFee': '0.0000', 'quoteGuaranteedExecutionFee': '0', 'halfS

AttributeError: 'list' object has no attribute 'get'